<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#쿠팡-데이터-크롤링" data-toc-modified-id="쿠팡-데이터-크롤링-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>쿠팡 데이터 크롤링</a></span></li></ul></div>

## 쿠팡 데이터 크롤링

In [42]:
import re
import datetime as dt
import pandas as pd
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException


from tqdm import tqdm
import time

In [45]:
#url = 'https://www.coupang.com/vp/products/6079767106?itemId=857429009&vendorItemId=5175887842&q=%EB%B9%84%EB%B9%84%EA%B3%A0%EB%A7%8C%EB%91%90&itemsCount=36&searchId=917d4128f6fe431fb423dde68a176f49&rank=8&isAddedCart='
#url = 'https://www.coupang.com/vp/products/11663052?itemId=18492743084&vendorItemId=86057057964&q=%EB%B9%84%EB%B9%84%EA%B3%A0+%EB%A7%8C%EB%91%90&itemsCount=36&searchId=ae381b42fa2149d0a14fb4571bc97d04&rank=3&isAddedCart='
# url = 'https://www.coupang.com/vp/products/7525648896?itemId=19747570159&vendorItemId=86851138379&pickType=COU_PICK&q=%ED%95%B4%ED%83%9C+%EB%A7%8C%EB%91%90&itemsCount=36&searchId=68d7ca177daf4390ba27874f7c690b55&rank=1&isAddedCart='
url = 'https://www.coupang.com/vp/products/317713654?itemId=1013591812&vendorItemId=5448197656&pickType=COU_PICK&q=%EB%8F%99%EC%9B%90+%EB%A7%8C%EB%91%90&itemsCount=36&searchId=80a42d50bf214711aeebf4891ea455ce&rank=2&isAddedCart='

In [49]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))


for rating in range(3,5):
    
    driver.get(url) #해당 url 열기, 수정
    driver.implicitly_wait(10)

    for _ in range(5):
        # 스크롤 다운
        time.sleep(1)
        driver.find_element(by=By.TAG_NAME, value='body').send_keys(Keys.END)
    
    # 상품평 버튼 클릭
    reviews_button = driver.find_element(By.NAME, 'review')
    reviews_button.click()

    time.sleep(5)
    
    #데이터 저장을 위한 기본 데이터프레임 생성
    dataframe = pd.DataFrame({'Title':[], 'Review':[], 'Reviewer':[], 'Date':[],'Star':[], 'Images':[],'Question1' : [],'Answer1':[],'Question2' : [],'Answer2':[],'Help':[]})

    # 별점 필터링 버튼 클릭
    ratings_button = driver.find_element(By.CLASS_NAME, 'sdp-review__article__order__star__all__current__all-content')  # 원하는 버튼의 클래스 이름으로 선택자를 변경
    ratings_button.click()
    
    #html 긁어오기
    html0 = driver.page_source
    html = BeautifulSoup(html0,'html.parser')

    # 1점부터 5점까지 별점 필터링
    driver.implicitly_wait(10)
    rating_button = driver.find_element(by = 'xpath', value = f'//*[@id="btfTab"]/ul[2]/li[3]/div/div[6]/section[2]/div[3]/div[2]/ul/li[{5 - rating}]')
    
    rating_button.click() 
    driver.implicitly_wait(10)
    
    #html 긁어오기
    html0 = driver.page_source
    html = BeautifulSoup(html0,'html.parser')


    page_num = 1
    while page_num < 1000:

        while True:
            try:
                # Locate the element (button in your case)
                page_button = driver.find_element(by='xpath', value=f'//*[@id="btfTab"]/ul[2]/li[3]/div/div[6]/section[4]/div[3]/button[{((page_num-1) % 10) + 2}]')
                
                # Click the button
                page_button.click()

                # If click is successful, break out of the loop
                break

            except StaleElementReferenceException:
                # If StaleElementReferenceException occurs, wait for a few seconds and try again
                time.sleep(5)
                continue
            
            except NoSuchElementException:
                print('추출 완료')
                page_num = 2000
                # If NoSuchElementException occurs, break the loop
                break
            
        
        #html 긁어오기
        html0 = driver.page_source
        html = BeautifulSoup(html0,'html.parser')
                                   
        driver.implicitly_wait(20)
        
        rc = 0
        while rc < 5:
            try:
                whole = html.find_all('article', {'class':'sdp-review__article__list js_reviewArticleReviewList'})[rc]
            except:
                break
                
            # 타이틀이 없는 경우
            try:
                title = whole.find('div', {'class':"sdp-review__article__list__headline"}).get_text()
            except:
                title = -1
                
            # 리뷰 내용이 없는 경우
            try:
                review = whole.find_all('div',{"class" : "sdp-review__article__list__review__content js_reviewArticleContent"})[0].get_text() 
            except:
                review = -1
            
            reviewer = whole.find_all('span',{"class" : "sdp-review__article__list__info__user__name js_reviewUserProfileImage"})[0].get_text()

            star_element = whole.find_all('div',{'class':'sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue'})[0]
            star = star_element['data-rating']

            date = whole.find_all('div',{'class':'sdp-review__article__list__info__product-info__reg-date'})[0].get_text()

            images_element = whole.find_all('div', {'class':'sdp-review__article__list__attachment js_reviewArticleListGalleryContainer'})[0]
            images = len(images_element.find_all('div',{'class':'sdp-review__article__list__attachment__list'}))
            
            # 설문조사에 답이 없거나 혹은 1개만 답한 경우
            try:
                survey = whole.find_all('div', {'class':'sdp-review__article__list__survey'})[0]

                q1 = survey.find_all('span', {'class':'sdp-review__article__list__survey__row__question'})[0].get_text()
                a1 = survey.find_all('span',{'class':'sdp-review__article__list__survey__row__answer'})[0].get_text()
                
                try:
                    q2 = survey.find_all('span', {'class':'sdp-review__article__list__survey__row__question'})[1].get_text()
                    a2 = survey.find_all('span',{'class':'sdp-review__article__list__survey__row__answer'})[1].get_text()
                
                except:
                    q2, a2 = -1, -1
                    
            except:
                q1, q2, a1,a2 = -1, -1, -1, -1
            
            # 도움이 되었다는 평가가 없는 경우
            try:
                helps = whole.find_all('span', {'class':"js_reviewArticleHelpfulCount"})[0].get_text()
            except:
                helps = -1
            
            insert_data = pd.DataFrame({'Title':[title], 'Review':[review], 'Reviewer':[reviewer], "Date":[date],'Star':[star], 'Images':[images],'Question1':[q1],'Answer1':[a1],'Question2':[q2],'Answer2':[a2],'Help':[helps]})

            # insert_data DataFrame을 기존 DataFrame에 연결합니다.
            dataframe = pd.concat([dataframe, insert_data], ignore_index=True)
            
            rc += 1
            
        print(f"{page_num}추출 Yes\n")  # 코드 실행 중 가시화를 위한 코드

        try:
            if page_num % 10 == 0:  # 페이지가 10의 배수 단위라면 다음 페이지 버튼을 눌러 페이지수 초기화
                nextpage_button = driver.find_element(by='xpath', value=f'//*[@id="btfTab"]/ul[2]/li[3]/div/div[6]/section[4]/div[3]/button[12]')
                nextpage_button.click()
                for _ in range(2):
                    # 스크롤 다운
                    driver.find_element(by=By.TAG_NAME, value='body').send_keys(Keys.END)
                    time.sleep(1)
                reviews_button.click()
                time.sleep(3)
        except:
            break
            
        page_num += 1

    #dataframe.to_excel(f'./data/풀무원_best_star{rating+1}.xlsx')
    #dataframe.to_excel(f'./data/비비고_best_star{rating+1}.xlsx')
    #dataframe.to_excel(f'./data/해태_best_star{rating+1}.xlsx')
    dataframe.to_excel(f'./data/동원_best_star{rating+1}.xlsx')



1추출 Yes

2추출 Yes

3추출 Yes

4추출 Yes

5추출 Yes

6추출 Yes

7추출 Yes

8추출 Yes

9추출 Yes

10추출 Yes

11추출 Yes

12추출 Yes

13추출 Yes

14추출 Yes

15추출 Yes

16추출 Yes

17추출 Yes

18추출 Yes

19추출 Yes

20추출 Yes

21추출 Yes

22추출 Yes

23추출 Yes

24추출 Yes

25추출 Yes

26추출 Yes

27추출 Yes

28추출 Yes

29추출 Yes

30추출 Yes

31추출 Yes

32추출 Yes

33추출 Yes

34추출 Yes

35추출 Yes

36추출 Yes

37추출 Yes

38추출 Yes

39추출 Yes

40추출 Yes

41추출 Yes

42추출 Yes

43추출 Yes

44추출 Yes

45추출 Yes

46추출 Yes

47추출 Yes

48추출 Yes

49추출 Yes

50추출 Yes

51추출 Yes

52추출 Yes

53추출 Yes

54추출 Yes

55추출 Yes

56추출 Yes

57추출 Yes

58추출 Yes

59추출 Yes

60추출 Yes

61추출 Yes

62추출 Yes

63추출 Yes

64추출 Yes

65추출 Yes

66추출 Yes

67추출 Yes

68추출 Yes

69추출 Yes

70추출 Yes

71추출 Yes

72추출 Yes

73추출 Yes

74추출 Yes

75추출 Yes

76추출 Yes

77추출 Yes

78추출 Yes

79추출 Yes

80추출 Yes

81추출 Yes

82추출 Yes

83추출 Yes

84추출 Yes

85추출 Yes

86추출 Yes

87추출 Yes

88추출 Yes

89추출 Yes

90추출 Yes

91추출 Yes

92추출 Yes

93추출 Yes

94추출 Yes

95추출 Yes

96추출 Yes

97추출 Yes

98추출 Yes

99추출 Yes

100추출 Yes

101추출 Ye

In [50]:
# dataframe.to_excel(f'./data/해태_best_star5.xlsx')
# dataframe

In [ ]:
# dataframe['Review'][20]
display(dataframe)
dataframe.to_excel('풀무원_best')

In [25]:
dataframe.to_excel(f'./data/비비고_best_star1.xlsx')